### 栅格数据与矢量数据的相互转换


In [1]:
from osgeo import gdal
from osgeo import osr, ogr
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
path_vec = 'data/Section-5/s2_chenggong_mndwi_polygon.shp'
path_raster = 'data/Section-5/s2_chenggong_mndwi_polygon2raster.tif'


### 主要参数：gdal.RasterizeLayer（dataset, bands, layer, burn_values, options）
dataset:输出栅格数据；bands:指定要更新的波段个数; layer:指定矢量图层；burn_values:输出图像的像素值；options:控制栅格化的参数。  
参考：https://gdal.org/api/python/osgeo.gdal.html 

In [3]:
from osgeo import gdal, ogr

# Define pixel_size and NoData value of new raster
pixel_size = 0.00018

# Open the data source and read in the extent
dset_vec = ogr.Open(path_vec)
source_layer = dset_vec.GetLayer()
x_min, x_max, y_min, y_max = source_layer.GetExtent()
geo_trans = (x_min, pixel_size, 0, y_max, 0, -pixel_size)
# 定义投影
wgs84 = osr.SpatialReference(); wgs84.ImportFromEPSG(4326)
wgs84_wkt = wgs84.ExportToWkt()

# Create the destination data source
x_size = int((x_max - x_min) / pixel_size)
y_size = int((y_max - y_min) / pixel_size)
dset_raster = gdal.GetDriverByName('GTiff').Create(path_raster, x_size, y_size, 1, gdal.GDT_Byte)
dset_raster.SetGeoTransform(geo_trans)
dset_raster.SetProjection(wgs84_wkt)
band = dset_raster.GetRasterBand(1)
band.SetNoDataValue(-999)
# Rasterize
gdal.RasterizeLayer(dataset=dset_raster, bands=[1], layer=source_layer, options=["ATTRIBUTE=DN"])
# gdal.RasterizeLayer(dataset=dset_raster, bands=[1], layer=source_layer, burn_values=[10])
dset_raster = None


### 快捷方式（控制台命令）：gdal_rasterize/gdal.Rasterize()
参考：https://gdal.org/programs/gdal_rasterize.html  



In [6]:
path_vec = 'data/Section-5/s2_chenggong_mndwi_polygon.shp'
path_raster = 'data/Section-5/s2_chenggong_mndwi_polygon2raster.tif'
!gdal_rasterize -b 1 -l s2_chenggong_mndwi_polygon $path_vec $path_raster -a DN


0...10...20...30...40...50...60...70...80...90...100 - done.


### 课后练习
在将矢量数据转为栅格数据中，使转换获得的栅格数据尺寸、分辨率、投影等信息与用于获得呈贡区域mndwi指数图像的原始遥感影像一致。

